In [60]:
import numpy as np

# Constants
DAYS = 5  # Monday to Friday
TIME_SLOTS_PER_DAY = 10  # 8 timeslots with breaks in between
MORNING_SLOTS = 6  # Morning session slots
AFTERNOON_SLOTS = 4  # Afternoon session slots
ROOMS = 10  # Number of available rooms
PROFESSORS = 6  # Number of professors
COURSES = 10  # Number of courses to be scheduled
SECTIONS = 20  # Number of sections
SECTION_SIZE = [60, 120]  # Section size for small room and large hall
MAX_COURSES_PER_PROFESSOR = 3
MAX_COURSES_PER_SECTION = 5

# Encoding parameters
BITS_PER_DAY = 3
BITS_PER_SLOT = 4  # Including breaks
BITS_PER_ROOM = 4
BITS_PER_PROFESSOR = 3
BITS_PER_COURSE = 4
BITS_PER_SECTION = 5

# Generate initial population
def generate_initial_population(pop_size):
    population = []
    for _ in range(pop_size):
        chromosome = ""
        # Generate chromosome for each course-section pair
        for _ in range(COURSES * SECTIONS):
            # Encode course, section, professor, and time slots
            chromosome += generate_chromosome_for_course()
        population.append(chromosome)
    return population

# Generate chromosome for a course-section pair
def generate_chromosome_for_course():
    course = np.random.randint(COURSES)
    section = np.random.randint(SECTIONS)
    professor = np.random.randint(PROFESSORS)
    day = np.random.randint(DAYS)
    if day < 3:
        slot = np.random.randint(MORNING_SLOTS)
    else:
        slot = np.random.randint(AFTERNOON_SLOTS) + MORNING_SLOTS
    room = np.random.randint(ROOMS)
    # Encode information into bits
    chromosome = f"{np.binary_repr(course, width=BITS_PER_COURSE)}"
    chromosome += f"{np.binary_repr(section, width=BITS_PER_SECTION)}"
    chromosome += f"{np.binary_repr(professor, width=BITS_PER_PROFESSOR)}"
    chromosome += f"{np.binary_repr(day, width=BITS_PER_DAY)}"
    chromosome += f"{np.binary_repr(slot, width=BITS_PER_SLOT)}"
    chromosome += f"{np.binary_repr(room, width=BITS_PER_ROOM)}"
    return chromosome

# Decode chromosome into timetable
def decode_chromosome(chromosome):
    timetable = np.zeros((COURSES, SECTIONS, DAYS, TIME_SLOTS_PER_DAY, ROOMS))
    idx = 0
    for course in range(COURSES):
        for section in range(SECTIONS):
            for _ in range(2):  # Two lectures per course
                course_info = chromosome[idx:idx+BITS_PER_COURSE]
                section_info = chromosome[idx+BITS_PER_COURSE:idx+BITS_PER_COURSE+BITS_PER_SECTION]
                professor_info = chromosome[idx+BITS_PER_COURSE+BITS_PER_SECTION:idx+BITS_PER_COURSE+BITS_PER_SECTION+BITS_PER_PROFESSOR]
                day_info = chromosome[idx+BITS_PER_COURSE+BITS_PER_SECTION+BITS_PER_PROFESSOR:idx+BITS_PER_COURSE+BITS_PER_SECTION+BITS_PER_PROFESSOR+BITS_PER_DAY]
                slot_info = chromosome[idx+BITS_PER_COURSE+BITS_PER_SECTION+BITS_PER_PROFESSOR+BITS_PER_DAY:idx+BITS_PER_COURSE+BITS_PER_SECTION+BITS_PER_PROFESSOR+BITS_PER_DAY+BITS_PER_SLOT]
                room_info = chromosome[idx+BITS_PER_COURSE+BITS_PER_SECTION+BITS_PER_PROFESSOR+BITS_PER_DAY+BITS_PER_SLOT:idx+BITS_PER_COURSE+BITS_PER_SECTION+BITS_PER_PROFESSOR+BITS_PER_DAY+BITS_PER_SLOT+BITS_PER_ROOM]
                
                print("Course Info:", course_info)
                print("Section Info:", section_info)
                print("Professor Info:", professor_info)
                print("Day Info:", day_info)
                print("Slot Info:", slot_info)
                print("Room Info:", room_info)
                print("Index:", idx)
                
                idx += BITS_PER_COURSE + BITS_PER_SECTION + BITS_PER_PROFESSOR + BITS_PER_DAY + BITS_PER_SLOT + BITS_PER_ROOM
                
                day = int(day_info, 2)
                slot = int(slot_info, 2)
                room = int(room_info, 2)
                timetable[course][section][day][slot][room] = 1
    return timetable




# Fitness function
def calculate_fitness(timetable):
    conflicts = 0
    # Hard constraints
    for course in range(COURSES):
        for section in range(SECTIONS):
            for day in range(DAYS):
                for slot in range(TIME_SLOTS_PER_DAY):
                    if np.sum(timetable[course][section][day][slot]) > 1:
                        conflicts += 1
    # Soft constraints
    # Example: Minimize floor traversal
    # You can add more soft constraints here
    return -conflicts

# Crossover operation
def crossover(parent1, parent2):
    crossover_point = np.random.randint(len(parent1))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Mutation operation
def mutate(chromosome):
    mutation_point = np.random.randint(len(chromosome))
    mutated_bit = '0' if chromosome[mutation_point] == '1' else '1'
    mutated_chromosome = chromosome[:mutation_point] + mutated_bit + chromosome[mutation_point+1:]
    return mutated_chromosome

# Genetic algorithm
def genetic_algorithm(population, num_generations):
    for generation in range(num_generations):
        # Decode chromosomes into timetables
        timetables = [decode_chromosome(chromosome) for chromosome in population]
        
        # Evaluate fitness
        fitness_scores = [calculate_fitness(timetable) for timetable in timetables]
        
        # Selection
        selected_indices = np.random.choice(len(population), size=len(population), replace=True, p=softmax(fitness_scores))
        selected_population = [population[i] for i in selected_indices]
        
        # Crossover
        new_population = []
        for i in range(0, len(selected_population), 2):
            if i + 1 < len(selected_population):
                child1, child2 = crossover(selected_population[i], selected_population[i+1])
                new_population.extend([child1, child2])
            else:
                new_population.append(selected_population[i])
        
        # Mutation
        mutation_rate = 0.01
        for i in range(len(new_population)):
            if np.random.random() < mutation_rate:
                new_population[i] = mutate(new_population[i])
        
        # Replace population
        population = new_population
        
        # Print best solution in current generation
        best_fitness = max(fitness_scores)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")
    
    # Return the best timetable after all generations
    best_timetable_index = np.argmax(fitness_scores)
    return timetables[best_timetable_index]

# Softmax function for selection
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# Main function
def main():
    # Generate initial population
    population = generate_initial_population(pop_size=100)
    
    # Run genetic algorithm
    best_timetable = genetic_algorithm(population, num_generations=100)
    
    # Print best timetable fitness
    best_fitness = calculate_fitness(best_timetable)
    print("Best Timetable Fitness:", best_fitness)
if __name__ == "__main__":
    #main()


SyntaxError: incomplete input (1892802255.py, line 174)

In [53]:
import random

# Define constants
DAYS = 5
TIME_SLOTS_PER_DAY = 2
ROOM_CAPACITY = {"Small": 60, "Large": 120}
NUM_ROOMS = 5  # Number of rooms
NUM_CLASSES = 15  # Define the number of classes
NUM_PROFESSORS = 5  # Define the number of professors
PROFESSORS = ["Prof1", "Prof2", "Prof3", "Prof4", "Prof5"][:NUM_PROFESSORS]  # Select from the list based on NUM_PROFESSORS
COURSES = ["Course1", "Course2", "Course3", "Course4", "Course5"][:NUM_CLASSES]  # Select from the list based on NUM_CLASSES
SECTIONS = ["SectionA", "SectionB", "SectionC", "SectionD", "SectionE"]
POP_SIZE = 100
NUM_GENERATIONS = 500
CROSSOVER_RATE = 0.7
MUTATION_RATE = 0.1

def generate_initial_population():
    population = []
    for _ in range(POP_SIZE):
        timetable = {}
        for course in COURSES:
            for section in SECTIONS:
                # Generate two distinct days for lectures
                day1, day2 = random.sample(range(DAYS), 2)
                # Ensure that the two days are not adjacent
                while abs(day1 - day2) <= 1:
                    day2 = random.randint(0, DAYS - 1)
                # Ensure that slots for lab lectures are consecutive
                slot1 = random.randint(0, TIME_SLOTS_PER_DAY - 2)  # Select first slot
                slot2 = slot1 + 1  # Second slot is consecutive to the first
                # Randomly select room capacity based on class size
                room_capacity = random.randint(40, 60) if random.choice([True, False]) else random.randint(100, 120)
                timetable[(course, section)] = {
                    "day1": day1,
                    "day2": day2,
                    "slot1": slot1,
                    "slot2": slot2,
                    "room": room_capacity,
                    "professor": random.choice(PROFESSORS)
                }
        population.append(timetable)
    return population


# Function to calculate fitness
def calculate_fitness(timetable):
    conflicts = 0
    professor_course_count = {professor: 0 for professor in PROFESSORS}  # Dictionary to store the count of courses for each professor
    section_course_count = {section: 0 for section in SECTIONS}  # Dictionary to store the count of courses for each section
    professor_schedule = {professor: set() for professor in PROFESSORS}  # Dictionary to store the schedule of each professor
    section_schedule = {section: set() for section in SECTIONS}  # Dictionary to store the schedule of each section
    for course1, info1 in timetable.items():
        professor = info1["professor"]
        professor_course_count[professor] += 1  # Increment the count of courses for the professor of course1
        section = course1[1]
        section_course_count[section] += 1  # Increment the count of courses for the section of course1
        professor_schedule[professor].add((info1["day1"], info1["slot1"]))  # Add the lecture schedule to the professor's schedule
        professor_schedule[professor].add((info1["day2"], info1["slot2"]))  # Add the lecture schedule to the professor's schedule
        section_schedule[section].add((info1["day1"], info1["slot1"], info1["room"]))  # Add the lecture schedule to the section's schedule
        section_schedule[section].add((info1["day2"], info1["slot2"], info1["room"]))  # Add the lecture schedule to the section's schedule
        for course2, info2 in timetable.items():
            if course1 != course2:
                # Check if the courses are on the same day and time slot
                if (info1["day1"], info1["slot1"]) == (info2["day1"], info2["slot1"]) or \
                   (info1["day2"], info1["slot2"]) == (info2["day2"], info2["slot2"]):
                    # Check if both courses belong to the same section
                    if course1[1] == course2[1]:
                        # Check if both courses are assigned to different rooms
                        if info1["room"] != info2["room"]:
                            conflicts += 1
    for count in professor_course_count.values():
        if count > 3:
            conflicts += count - 3  # Penalize if the course count for a professor exceeds 3
    for count in section_course_count.values():
        if count > 5:
            conflicts += count - 5  # Penalize if the course count for a section exceeds 5
    for professor_schedule_list in professor_schedule.values():
        for day_slot in professor_schedule_list:
            if list(professor_schedule_list).count(day_slot) > 1:
                conflicts += 1  # Penalize if a professor is assigned two different lectures at the same time
    for section_schedule_list in section_schedule.values():
        for day_slot_room in section_schedule_list:
            if list(section_schedule_list).count(day_slot_room) > 1:
                conflicts += 1  # Penalize if the same section is assigned to two different rooms at the same time
    for day in range(DAYS):
        for slot in range(TIME_SLOTS_PER_DAY):
            for room in ROOM_CAPACITY:
                room_occupancy = set()
                for course, info in timetable.items():
                    if info["day1"] == day and info["slot1"] == slot and info["room"] == room:
                        room_occupancy.add(course)
                    if info["day2"] == day and info["slot2"] == slot and info["room"] == room:
                        room_occupancy.add(course)
                if len(room_occupancy) > 1:
                    conflicts += 1
    return -conflicts

# Function for tournament selection
def tournament_selection(population, fitness_scores):
    selected_indices = random.sample(range(len(population)), k=2)
    if fitness_scores[selected_indices[0]] > fitness_scores[selected_indices[1]]:
        return population[selected_indices[0]]
    else:
        return population[selected_indices[1]]

# Function for crossover
def crossover(parent1, parent2):
    child = {}
    for course in COURSES:
        for section in SECTIONS:
            if random.random() < 0.5:
                child[(course, section)] = parent1[(course, section)]
            else:
                child[(course, section)] = parent2[(course, section)]
    return child

# Function for mutation
def mutate(timetable):
    for course in COURSES:
        for section in SECTIONS:
            if random.random() < MUTATION_RATE:
                timetable[(course, section)]["day1"] = random.randint(0, DAYS - 1)
                timetable[(course, section)]["day2"] = random.randint(0, DAYS - 1)
                timetable[(course, section)]["slot1"] = random.randint(0, TIME_SLOTS_PER_DAY - 1)
                timetable[(course, section)]["slot2"] = random.randint(0, TIME_SLOTS_PER_DAY - 1)
                timetable[(course, section)]["room"] = random.choice(list(ROOM_CAPACITY.keys()))
                timetable[(course, section)]["professor"] = random.choice(PROFESSORS)
    return timetable

# Main Genetic Algorithm function
def genetic_algorithm():
    population = generate_initial_population()
    for generation in range(NUM_GENERATIONS):
        fitness_scores = [calculate_fitness(timetable) for timetable in population]
        new_population = []
        for _ in range(POP_SIZE // 2):
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)
            if random.random() < CROSSOVER_RATE:
                offspring1 = crossover(parent1, parent2)
                offspring2 = crossover(parent1, parent2)
            else:
                offspring1 = parent1
                offspring2 = parent2
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))
        population = new_population
    best_timetable = max(population, key=lambda x: calculate_fitness(x))
    return best_timetable


# Function to convert slot number to time range
def slot_to_time(slot):
    if slot == 0:
        return "8:30 AM - 9:50 AM"
    elif slot == 1:
        return "10:00 AM - 11:20 AM"
    # Add more time slots as needed

def display_timetable(timetable):
    print("Course | Section | Day1 | Time1 | Room1 (Capacity) | Day2 | Time2 | Room2 (Capacity) | Professor")
    for (course, section), info in timetable.items():
        time1 = slot_to_time(info['slot1'])
        time2 = slot_to_time(info['slot2'])
        # Determine room capacity based on room size
        room_capacity = ROOM_CAPACITY["Small"] if info['room'] == "Small" else ROOM_CAPACITY["Large"]
        print(f"{course:6} | {section:7} | {info['day1']+1:4} | {time1:20} | {info['room']} ({room_capacity}) |"
              f" {info['day2']+1:4} | {time2:20} | {info['room']} ({room_capacity}) | {info['professor']:9}")

# Main function
def main():
    best_timetable = genetic_algorithm()
    print("Best Timetable:")
    display_timetable(best_timetable)

if __name__ == "__main__":
    main()


Best Timetable:
Course | Section | Day1 | Time1 | Room1 (Capacity) | Day2 | Time2 | Room2 (Capacity) | Professor
Course1 | SectionA |    5 | 8:30 AM - 9:50 AM    | Large (120) |    2 | 8:30 AM - 9:50 AM    | Large (120) | Prof1    
Course1 | SectionB |    2 | 8:30 AM - 9:50 AM    | Small (60) |    4 | 8:30 AM - 9:50 AM    | Small (60) | Prof4    
Course1 | SectionC |    1 | 8:30 AM - 9:50 AM    | Small (60) |    5 | 8:30 AM - 9:50 AM    | Small (60) | Prof5    
Course1 | SectionD |    1 | 10:00 AM - 11:20 AM  | Large (120) |    2 | 10:00 AM - 11:20 AM  | Large (120) | Prof5    
Course1 | SectionE |    1 | 8:30 AM - 9:50 AM    | Large (120) |    3 | 8:30 AM - 9:50 AM    | Large (120) | Prof5    
Course2 | SectionA |    2 | 10:00 AM - 11:20 AM  | Large (120) |    5 | 8:30 AM - 9:50 AM    | Large (120) | Prof5    
Course2 | SectionB |    5 | 10:00 AM - 11:20 AM  | Small (60) |    3 | 10:00 AM - 11:20 AM  | Small (60) | Prof3    
Course2 | SectionC |    2 | 10:00 AM - 11:20 AM  | Large (12

In [28]:
import random
import csv

# Define constants
DAYS = 5
TIME_SLOTS_PER_DAY = 4
ROOM_CAPACITY = {"Small": 60, "Large": 120}
NUM_ROOMS = 5  # Number of rooms
NUM_CLASSES = 15  # Define the number of classes
NUM_PROFESSORS = 5  # Define the number of professors
PROFESSORS = ["Prof1", "Prof2", "Prof3", "Prof4", "Prof5"][:NUM_PROFESSORS]  # Select from the list based on NUM_PROFESSORS
COURSES = ["Course1", "Course2", "Course3", "Course4", "Course5"][:NUM_CLASSES]  # Select from the list based on NUM_CLASSES
SECTIONS = ["SectionA", "SectionB", "SectionC", "SectionD", "SectionE"]
POP_SIZE = 10
NUM_GENERATIONS = 10
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.9



In [ ]:
def encode_individual(timetable, professors, theory_courses, lab_courses, sections, section_strengths):
    binary_individual = ""
    for entry in timetable:
        course_type, section, professor, day, timeslot, room = entry[1:]

        if course_type == 'Theory':
            course_index = theory_courses.index(course_type)
            professor_index = professors.index(professor)
            day_index = DAYS.index(day)
            timeslot_index = TIMESLOTS.index(timeslot)
            room_size_index = 1 if room == LARGE_HALL_SIZE else 0

            binary_individual += format(course_index, f'0{NUM_BITS_COURSE}b')
            binary_individual += format(professor_index, f'0{NUM_BITS_PROFESSOR}b')
            binary_individual += format(day_index, f'0{NUM_BITS_DAY}b')
            binary_individual += format(timeslot_index, f'0{NUM_BITS_TIMESLOT}b')
            binary_individual += format(room_size_index, f'0{NUM_BITS_ROOM_SIZE}b')

        elif course_type == 'Lab':
            course_index = lab_courses.index(course_type)
            professor_index = professors.index(professor)
            day_index = DAYS.index(day)
            room_size_index = 1 if room == LARGE_HALL_SIZE else 0

            binary_individual += format(course_index, f'0{NUM_BITS_COURSE}b')
            binary_individual += format(professor_index, f'0{NUM_BITS_PROFESSOR}b')
            binary_individual += format(day_index, f'0{NUM_BITS_DAY}b')
            binary_individual += format(room_size_index, f'0{NUM_BITS_ROOM_SIZE}b')

    return binary_individual


In [22]:

def generate_initial_population():
    population = []
    for _ in range(POP_SIZE):
        timetable = {}
        for course in COURSES:
            for section in SECTIONS:
                # Generate two distinct days for lectures
                day1, day2 = random.sample(range(DAYS), 2)
                # Ensure that the two days are not adjacent
                while abs(day1 - day2) <= 1:
                    day2 = random.randint(0, DAYS - 1)
                # Ensure that slots for lab lectures are consecutive
                slot1 = random.randint(0, TIME_SLOTS_PER_DAY - 2)  # Select first slot
                slot2 = slot1 + 1  # Second slot is consecutive to the first
                # Randomly select room capacity based on class size
                room_size = random.choice(["Small", "Large"])
                room_capacity = random.randint(40, 60) if room_size == "Small" else random.randint(100, 120)
                timetable[(course, section)] = {
                    "Course": course,
                    "Theory/Lab": "Lab" if random.random() < 0.5 else "Theory",
                    "Section": section,
                    "Section-Strength": room_capacity,
                    "Professor": random.choice(PROFESSORS),
                    "First-lecture-day": day1,
                    "First-lecture-timeslot": slot1,
                    "First-lecture-room": random.randint(1, NUM_ROOMS),
                    "First-lecture-room-size": room_size,
                    "Second-lecture-day": day2,
                    "Second-lecture-timeslot": slot2,
                    "Second-lecture-room": random.randint(1, NUM_ROOMS),
                    "Second-lecture-room-size": room_size
                }
        population.append(timetable)
    return population


In [23]:


# Function to calculate fitness
def calculate_fitness(timetable):
    conflicts = 0
    professor_course_count = {professor: 0 for professor in PROFESSORS}  # Dictionary to store the count of courses for each professor
    section_course_count = {section: 0 for section in SECTIONS}  # Dictionary to store the count of courses for each section
    professor_schedule = {professor: set() for professor in PROFESSORS}  # Dictionary to store the schedule of each professor
    section_schedule = {section: set() for section in SECTIONS}  # Dictionary to store the schedule of each section
    for course1, info1 in timetable.items():
        professor = info1["Professor"]  # Updated key from "professor" to "Professor"
        professor_course_count[professor] += 1  # Increment the count of courses for the professor of course1
        section = course1[1]
        section_course_count[section] += 1  # Increment the count of courses for the section of course1
        professor_schedule[professor].add((info1["First-lecture-day"], info1["First-lecture-timeslot"]))  # Add the lecture schedule to the professor's schedule
        professor_schedule[professor].add((info1["Second-lecture-day"], info1["Second-lecture-timeslot"]))  # Add the lecture schedule to the professor's schedule
        section_schedule[section].add((info1["First-lecture-day"], info1["First-lecture-timeslot"], info1["First-lecture-room"]))  # Add the lecture schedule to the section's schedule
        section_schedule[section].add((info1["Second-lecture-day"], info1["Second-lecture-timeslot"], info1["Second-lecture-room"]))  # Add the lecture schedule to the section's schedule
        for course2, info2 in timetable.items():
            if course1 != course2:
                # Check if the courses are on the same day and time slot
                if (info1["First-lecture-day"], info1["First-lecture-timeslot"]) == (info2["First-lecture-day"], info2["First-lecture-timeslot"]) or \
                   (info1["Second-lecture-day"], info1["Second-lecture-timeslot"]) == (info2["Second-lecture-day"], info2["Second-lecture-timeslot"]):
                    # Check if both courses belong to the same section
                    if course1[1] == course2[1]:
                        # Check if both courses are assigned to different rooms
                        if info1["First-lecture-room"] != info2["First-lecture-room"] or \
                           info1["Second-lecture-room"] != info2["Second-lecture-room"]:
                            conflicts += 1
    for count in professor_course_count.values():
        if count > 3:
            conflicts += count - 3  # Penalize if the course count for a professor exceeds 3
    for count in section_course_count.values():
        if count > 5:
            conflicts += count - 5  # Penalize if the course count for a section exceeds 5
    for professor_schedule_list in professor_schedule.values():
        for day_slot in professor_schedule_list:
            if list(professor_schedule_list).count(day_slot) > 1:
                conflicts += 1  # Penalize if a professor is assigned two different lectures at the same time
    for section_schedule_list in section_schedule.values():
        for day_slot_room in section_schedule_list:
            if list(section_schedule_list).count(day_slot_room) > 1:
                conflicts += 1  # Penalize if the same section is assigned to two different rooms at the same time
    for day in range(DAYS):
        for slot in range(TIME_SLOTS_PER_DAY):
            for room in ROOM_CAPACITY:
                room_occupancy = set()
                for course, info in timetable.items():
                    if info["First-lecture-day"] == day and info["First-lecture-timeslot"] == slot and info["First-lecture-room"] == room:
                        room_occupancy.add(course)
                    if info["Second-lecture-day"] == day and info["Second-lecture-timeslot"] == slot and info["Second-lecture-room"] == room:
                        room_occupancy.add(course)
                if len(room_occupancy) > 1:
                    conflicts += 1
    return -conflicts


In [24]:
# Function for tournament selection
def tournament_selection(population, fitness_scores):
    selected_indices = random.sample(range(len(population)), k=2)
    if fitness_scores[selected_indices[0]] > fitness_scores[selected_indices[1]]:
        return population[selected_indices[0]]
    else:
        return population[selected_indices[1]]

# Function for crossover
def crossover(parent1, parent2):
    child = {}
    for course in COURSES:
        for section in SECTIONS:
            if random.random() < 0.5:
                child[(course, section)] = parent1[(course, section)]
            else:
                child[(course, section)] = parent2[(course, section)]
    return child

# Function for mutation
def mutate(timetable):
    for course in COURSES:
        for section in SECTIONS:
            if random.random() < MUTATION_RATE:
                timetable[(course, section)]["day1"] = random.randint(0, DAYS - 1)
                timetable[(course, section)]["day2"] = random.randint(0, DAYS - 1)
                timetable[(course, section)]["slot1"] = random.randint(0, TIME_SLOTS_PER_DAY - 1)
                timetable[(course, section)]["slot2"] = random.randint(0, TIME_SLOTS_PER_DAY - 1)
                timetable[(course, section)]["room"] = random.choice(list(ROOM_CAPACITY.keys()))
                timetable[(course, section)]["professor"] = random.choice(PROFESSORS)
    return timetable



In [25]:

def genetic_algorithm():
    population = generate_initial_population()
    for generation in range(NUM_GENERATIONS):
        fitness_scores = [calculate_fitness(timetable) for timetable in population]
        # Track the best fitness rate in the current generation
        best_fitness = max(fitness_scores)
        print(f"Generation {generation + 1}: Best Fitness Rate = {best_fitness}")
        new_population = []
        for _ in range(POP_SIZE // 2):
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)
            if random.random() < CROSSOVER_RATE:
                offspring1 = crossover(parent1, parent2)
                offspring2 = crossover(parent1, parent2)
            else:
                offspring1 = parent1
                offspring2 = parent2
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))
        population = new_population
    best_timetable = max(population, key=lambda x: calculate_fitness(x))
    return best_timetable


In [26]:


# Function to convert slot number to time range
def slot_to_time(slot):
    if slot == 0:
        return "8:30 AM - 9:50 AM"
    elif slot == 1:
        return "10:00 AM - 11:20 AM"
    elif slot == 2:
        return "11:30 AM - 12:50 PM"
    elif slot == 3:
        return "01:00 PM - 02:20 PM"
    # Add more time slots as needed

def save_timetable_to_csv(timetable, filename):
    # Extract unique number of columns
    columns = set()
    for _, info in timetable.items():
        columns.update(info.keys())
    columns = list(columns)
    
    # Write CSV
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(columns)  # Write header
        for (_, section), info in timetable.items():
            row = [section]  # First column is section
            for column in columns[1:]:  # Exclude section from columns
                if column.endswith("time"):  # Convert slot number to time range
                    row.append(slot_to_time(info.get(column, "")))
                else:
                    row.append(info.get(column, ""))  # Add data or empty string if not present
            writer.writerow(row)


In [27]:

def main():
    best_timetable = genetic_algorithm()
    save_timetable_to_csv(best_timetable, "timetable.csv")
    
if __name__ == "__main__":
    main()

Generation 1: Best Fitness Rate = -12
Generation 2: Best Fitness Rate = -12
Generation 3: Best Fitness Rate = -11
Generation 4: Best Fitness Rate = -12
Generation 5: Best Fitness Rate = -10
Generation 6: Best Fitness Rate = -10
Generation 7: Best Fitness Rate = -10
Generation 8: Best Fitness Rate = -10
Generation 9: Best Fitness Rate = -12
Generation 10: Best Fitness Rate = -11
